In [63]:

from os.path import exists
import numpy as np
#get_self: n_jobs,n_machines
instance_path = "/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/standard/abz5.txt"
if not exists(instance_path):
    raise FileExistsError(f"File does not exists or {instance_path} is no valid path")
if instance_path is None:
    raise ValueError("no instance is given")
    # here begins instance parser
if any(x in instance_path for x in ["abz","dmu","yn","ta","swv","orb","la","ft"]):
    n_line = 0
    with open(instance_path, 'r') as text:
        for line_txt in text.readlines():
            n_line+=1
            if n_line == 1:
                n_jobs,n_machines=map(int,line_txt.split())
                # matrix contains machine and coresponding job lenth
                job_machine_matrix = np.zeros((n_jobs,n_machines), dtype=(np.int32))
                job_tasklength_matrix=np.zeros((n_jobs,n_machines), dtype=(np.int32))
                # contains time to complete jobs
                job_length = np.zeros(n_jobs, dtype=np.int32)
            else:
                j=0
                nr_job=n_line-1
                cum_jobtime=0
                for i in line_txt.split("	"):
                    i=int(i)
                    if j%2==0:
                        job_machine_matrix[nr_job-1][int((j)/2)]= i 
                    else:
                        job_tasklength_matrix[nr_job-1][int((j-1)/2)]= i 
                        cum_jobtime+=i
                    j+=1    
                job_length[nr_job-1]= cum_jobtime
        #todo: implement more conventions, e.g. IMA or Taillard
else:
    raise NotImplementedError("till now only standard instances are implemented")


observation_space_shape=(max(n_jobs+1,n_machines),6)    
import gym
action_space = gym.spaces.Discrete(n_jobs+1) 
# observation space contains for every job:
    #   - task can be assigned (previouse task of job is finished! and machine is free -> legal action mask)
    #   - process time for next task // normalized by longest task over all jobs
    #   - seconds this job needs in best case // normalized by longest job

    #   - earliest time next task from different job needs this machine
    #   - smth. like % of next task or duration of task

# observation space contains for every machine:
    #   - time which this machine has to be running in total // normalized by all jobs: beginning-> 1, finish ->0
    #   -             
observation_space = gym.spaces.Dict({
        "action_mask": gym.spaces.Box(0, 1, shape=(action_space.n,), dtype=np.int32),
        "obs": gym.spaces.Box(low=0.0,high=1.0,
            shape=observation_space_shape,dtype=np.float64)   
                                                            }
                                                )
    


In [64]:


from math import ceil
import math

def norm_with_max(value,max_value):
    # just a function to norm towards a max value to use the same round metric and to get values between 0 and 1
    return round(value/max_value,4)

def denorm_with_max(normed_value,max_value):
    return ceil(normed_value*max_value)


def get_legal_actions(obs):
    action_mask=np.full((n_jobs+1,),False)
    avail_machines=[]
    for i in range(n_machines):
        print(current_machines_status[i][0])
        if math.isnan(current_machines_status[i][0]):
            avail_machines.append(i)
    print("avail machines:")
    print(avail_machines)
    for i in range(n_jobs):
        #obs i 5 is normed count of finished tasks
        if job_machine_matrix[i][denorm_with_max(obs[i][5],n_tasks)] in avail_machines:
            action_mask[i]=True

    # definition for timestep skipping, by now just skip if there is no free machine for current job, 
    # todo: define when it is necessary to skip jobs.
    if True not in action_mask:
        action_mask[n_jobs]=True

    return action_mask

count_finished_tasks_job_matrix=np.zeros(n_jobs,dtype=np.int32)
current_timestep=0
longest_tasklength=np.amax(job_tasklength_matrix)
n_tasks=job_tasklength_matrix.shape[1]
# first entry stores active processed time of job, second entry overall task time
processed_and_max_time_job_matrix=np.zeros((n_jobs,2))
# machine status stores: current job, time operation/task left, total time working
current_machines_status=np.zeros((n_machines,3))
# set current job to nan because no job is assigned to the machines
current_machines_status[:,0]=np.nan

for i in range(n_jobs):
    processed_and_max_time_job_matrix[i][1]=sum(job_tasklength_matrix[i])


# observation space contains for every job:
            #   - task can be assigned (previouse task of job is finished! and machine is free -> legal action mask)
            #   - process time for next task // normalized by longest task over all jobs
            #   - seconds this job needs in best case // normalized by longest job

            #   - earliest time next task from different job needs this machine
            #   - smth. like % of next task or duration of task

        # observation space contains for every machine:
            #   - time which this machine has to be running in total // normalized by all jobs: beginning-> 1, finish ->0
            #   -             
"""
    -Legal job
            -Left over time on the current op
            -Current operation %
            -Total left over time
            -When next machine available
            -Time since IDLE: 0 if not available, time otherwise
            -Total IDLE time in the schedule
"""

# inital observation
observation= np.zeros(shape=observation_space_shape)
# first row is legal action mask
observation[:,0]=np.full((n_jobs+1,),True)
# set skip timestep to False
observation[n_jobs][0]=False


for i in range(n_jobs):
    # second attribute: current time left on current task:
    observation[i,1]=norm_with_max(0,longest_tasklength)
    # third attribute: process time of next scheduled task
    observation[i,2]=norm_with_max(job_tasklength_matrix[i][count_finished_tasks_job_matrix[i]],longest_tasklength)
    #fourth attribute: process on current job in percent
    observation[i,3]=norm_with_max(processed_and_max_time_job_matrix[i][0],processed_and_max_time_job_matrix[i][1])
    # count finished tasks// normalized
    observation[i][4]=norm_with_max(count_finished_tasks_job_matrix[i],n_tasks)
for i in range(n_machines):
    # time to next machine available
    observation[i][5]=norm_with_max(current_machines_status[i][1],longest_tasklength)

    




In [65]:
import math
"""
for i in range(n_jobs):
        #obs i 5 is normed count of finished tasks
        if job_machine_matrix[i][denorm_with_max(count_finished_tasks_job_matrix[i],n_tasks)] in avail_machines:
            action_mask[i]=True
    observation[:,0]=action_mask
    for i in range(n_jobs):

        # third attribute: process time of next scheduled task
        observation[i,2]=norm_with_max(job_tasklength_matrix[i][count_finished_tasks_job_matrix[i]],longest_tasklength)
        #fourth attribute: process on current job in percent
        observation[i,3]=norm_with_max(processed_and_max_time_job_matrix[i][0],processed_and_max_time_job_matrix[i][1])
        # count finished tasks// normalized
        observation[i][4]=norm_with_max(count_finished_tasks_job_matrix[i],n_tasks)

    observation[:,1]=np.zeros(n_jobs)    
    for i in range(n_machines):
        # time to next machine available
        observation[i][5]=norm_with_max(current_machines_status[i][1],longest_tasklength)
        # second attribute: current time left on current task, if task is not assigned it got value 0: todo: checkout it 0 or full time makes sense
        observation[current_machines
"""
def observation_to_state(obs):
    #todo: complete function
    state=np.concatenate(np.ravel(obs[0:n_jobs+1,0]),np.ravel(obs[0:n_jobs,1:4]),np.ravel(obs[0:n_machines,5]),axis=None)
    return state
# stores all timestemps
timestemp_list=[]
done=False
#stores quatuple(job,task,start_time,finish_time) 
production_list=np.empty((n_machines,n_tasks),dtype=object)

#
count_finished_tasks_machine_matrix=np.zeros(n_machines)
# starting point
timestemp_list.append(0)


def step(action):
    reward=0
    if observation[action,0]==False:
        raise ValueError("action is not in legal actions: implement to do nothing...")
    if action==n_jobs:
        #implement to jump to next timestemp
        #check if done? todo: add aditional checks like are all tasks done and satisfied or just problems with timestemp?
        if not timestemp_list:
            # this is reward to mini makespan: todo:implement utilisation etc.
            reward=-current_timestep
            done=True
        else:
            # temp. saving timestep
            forward_timestep=current_timestep
            timestemp_list.sort()
            current_timestep=timestemp_list.pop(0)
            # how large is the timestep?
            forward_timestep=current_timestep-forward_timestep
            # substract forwarded time from current production, update on all active machines:
            for i in range(n_machines):
                #check if machine is active, active if 0th entry is not nan
                if math.isnan(current_machines_status[i,0])==False:
                    job=current_machines_status[i,0]
                    processed_and_max_time_job_matrix[job,1]=processed_and_max_time_job_matrix[job,1]+forward_timestep
                    # leftover time for task on machine i
                    current_machines_status[i,1]-=forward_timestep
                    #total time of machine working
                    current_machines_status[i,2]+=forward_timestep
                    if processed_and_max_time_job_matrix[job,1] >= processed_and_max_time_job_matrix[job,1]:
                        #machine is finished with task, need reward before
                        current_machines_status[i,0]=np.nan
                        # residual time of job, obviously =0
                        if current_machines_status[i,1]!=0:
                            raise ValueError("Problems with left over time for task") 
                        count_finished_tasks_job_matrix[job]+=1
                        count_finished_tasks_machine_matrix[i]+=1 
    else:        
        # assign task (from action) to available machine and update the machine status
        #set task length:
        processed_and_max_time_job_matrix[action,1]=job_tasklength_matrix[action,count_finished_tasks_job_matrix[action]]
        processed_and_max_time_job_matrix[action,1]=0
        # block the free machine with current job
        #todo: delete error
        if math.isnan(current_machines_status[job_machine_matrix[action,count_finished_tasks_job_matrix[action]],0])==False:
            raise ValueError("machine is not free") 
        current_machines_status[job_machine_matrix[action,count_finished_tasks_job_matrix[action]],0]=action
        current_machines_status[job_machine_matrix[action,count_finished_tasks_job_matrix[action]],1]=job_tasklength_matrix[action,count_finished_tasks_job_matrix[action]]
        # update production_list:(job,tas,start,finish)
        production_list[job_machine_matrix[action,count_finished_tasks_job_matrix[action]],count_finished_tasks_job_matrix[job_machine_matrix[action,count_finished_tasks_job_matrix[action]]]]=(action,count_finished_tasks_job_matrix[action],current_timestep,current_timestep+job_tasklength_matrix[action,count_finished_tasks_job_matrix[action]])
        # add end time to timestemp list, first check if value is not already in list:
        if current_timestep+job_tasklength_matrix[action,count_finished_tasks_job_matrix[action]] not in timestemp_list:
            timestemp_list.append(current_timestep+job_tasklength_matrix[action,count_finished_tasks_job_matrix[action]])
    # update observation:

    action_mask=np.full((n_jobs+1,),False)
    avail_machines=[]
    for i in range(n_machines):
        if math.isnan(current_machines_status[i][0]):
            avail_machines.append(i)
    for i in range(n_jobs):
        #obs i 5 is normed count of finished tasks
        if job_machine_matrix[i][denorm_with_max(count_finished_tasks_job_matrix[i],n_tasks)] in avail_machines:
            action_mask[i]=True
    observation[:,0]=action_mask
    for i in range(n_jobs):

        # third attribute: process time of next scheduled task
        observation[i,2]=norm_with_max(job_tasklength_matrix[i][count_finished_tasks_job_matrix[i]],longest_tasklength)
        #fourth attribute: process on current job in percent
        observation[i,3]=norm_with_max(processed_and_max_time_job_matrix[i][0],processed_and_max_time_job_matrix[i][1])
        # count finished tasks// normalized
        observation[i][4]=norm_with_max(count_finished_tasks_job_matrix[i],n_tasks)

    observation[:,1]=np.zeros(n_jobs)    
    for i in range(n_machines):
        # time to next machine available
        observation[i][5]=norm_with_max(current_machines_status[i][1],longest_tasklength)
        # second attribute: current time left on current task, if task is not assigned it got value 0: todo: checkout it 0 or full time makes sense
        observation[current_machines_status[i,0],1]=norm_with_max(current_machines_status[i][1],longest_tasklength)
    info="everything fine"
    state=observation_to_state(observation)
    return state, reward, done, info

    

        

        

In [66]:
action_mask=get_legal_actions(observation)
production_list=np.empty((n_machines,n_tasks),dtype=object)
x=(1,2,3,4)
production_list[1,1]=x



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
avail machines:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [67]:
current_machines_status=np.zeros((n_machines,3))
# set current job to nan because no job is assigned to the machines
current_machines_status[:,0]=np.nan

In [68]:
print(job_tasklength_matrix.shape[1])
max(n_jobs+1,n_machines)

10


11

In [69]:
timestemp_list=[1,5,2,3,4,5]
print(timestemp_list)
tmp=timestemp_list.sort()
print(timestemp_list)
x=timestemp_list.pop(1)
print(x)
print(timestemp_list)
print(tmp)

[1, 5, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 5]
2
[1, 3, 4, 5, 5]
None


In [70]:
y=(np.ravel(observation[0:n_jobs+1,0:2]))
z=np.ravel(observation[0:n_jobs,1])
x=np.concatenate((x,z),axis=None)
print(y)
print(z)
print(x)


[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
def render():

    production_list


In [3]:
import plotly.figure_factory as ff

df = [dict(Task="Job-1", Start='0', Finish='20', Resource='Complete'),
      dict(Task="Job-1", Start='25', Finish='30', Resource='Incomplete'),
      dict(Task="Job-2", Start='10', Finish='40', Resource='Not Started'),
     ]

colors = {'Not Started': 'rgb(220, 0, 0)',
          'Incomplete': (1, 0.9, 0.16),
          'Complete': 'rgb(0, 255, 100)'}

fig = ff.create_gantt(df, colors=colors, index_col='Resource', show_colorbar=True,
                      group_tasks=True)
fig.show()